### Exercice 4. Transient Flow in 1D

The goal of this exercise is to code and understand a finite difference / finite volume method, solving the problem of one-dimensional terzaghi flow.

In [2]:
# importing the necessary python modules
# numpy and matplotlib are popular packages for scientific computing and plotting
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# interactive python (iPython) magic to instruct jupyter how to show the plots with respect to the code cells
%matplotlib inline

# scipy is an extension for numpy, used extensively in scientific computing
import scipy as sp

## One dimensional diffusion

We can imagine a segment of length $2L$ under uniform pore pressure $p_0$. At $t=t_0$, the segment is drained from both ends. Assuming symmetry at $x=L$, we can write the problem for $x \isin [0, L]$ with this one dimensional differential equation:

$$ \frac{\partial p}{\partial t} - c\frac{\partial^2 p}{\partial x^2}=0, $$

also using those boundary conditions:
$$p(x=0, t>0) = 0,$$
$$\frac{\partial p}{\partial x}(x = L, t) = 0$$
and
$$p(x, t=0) = p_0.$$


We will start by defining a very simple one dimensional mesh, which we will use to compute finite differences.

In [ ]:
# Mesh definition
# Complete below
# L  =      # length of the domain
# p0 =      # initial pressure
# c  =      # consolidation coefficient
n  = 100    # number of elements

x = np.linspace(0, L, n + 1) # this function of numpy gives you an evenly spaced array
                             # between 0 and L with n + 1 elements (including endpoints)

# Complete below
# dx =       # dx is the spacing between two nodes, i.e., the cell size


print(f'The array of the node coordinates is: {x=}')
print(f'with a cell size of {dx=}')

### Finite difference scheme

We can compute the solution to this differential equation as a system of equations where we are discretizing the pressure gradient into a ratio between the pressure difference and the distance between the measurement points.

We do so by building the matrix $\mathbb{L}$. Take care at the boundaries where you have to adapt the matrix to validate the boundary conditions.

In [ ]:
# We first initiate a sparse matrix.
Lmat = sp.sparse.csc_array((n, n))

# We then iterate over all the nodes and assign the corresponding entries.
# Complete within the loop
for i in range(n):
    # Lmat[ , ] =       # Value at the diagonal

    # Off diagonal values depending on the boundary conditions
    # Lmat[ , ] = 
    # Lmat[ , ] = 

# Applying the spatial difference
# Complete below
# Lmat = 

# We can now inspect the entries in the matrix
plt.imshow(Lmat.todense())

After the creation of the $\mathbb{L}$ matrix. we are ready to solve the system.

In your parameter choice recall that $\theta = 0$ gives you a forward Euler (explicit) scheme wherease a $\theta$ of $1$ is equivalent to a backward Euler (implicit) scheme.

In [ ]:
# We define the remaining parameters of the problem

theta = 0.8 
# complete below
# dt =  # Recall to take care of the CFL condition

# As we are using a fixed time step dt, we can now pre-define our evaluation times
t = np.arange(0, 1 + dt, dt)
# Lets print the characteristics of our time integration scheme.
print(f'We perform a total of {len(t)=} timesteps.')
print(f'with a timestep of {dt=}')

# We now pre-define the solution vector, where we have one line for every timestep.
heads = np.zeros((len(t), n))

# Complete this array with the initial solution
# heads[0] = 

# And also create the identity matrix.
I = sp.sparse.eye(n)

# We observe that the left hand side of our equation is independent on the time and
# current solution, we thus calculate it beforehand.

# complete below

# left = 

leftInv = sp.sparse.linalg.splu(left)

# Finally we iterate over the timesteps and calculate the change in pressure to
# obtain the new head
for i in range(len(t) - 1):
    # complete the equation
    # dp = leftInv.solve(...)
    # heads[i + 1] = 

### Comparing with an analytical solution

The analytical solution for the pressure $p(x, t)$ is given by the following equation:

$$ p(x, t) = p_0 \sum_{k=1, 3, ..}^{\infty}\frac{4}{k\pi}\sin\left(\frac{k\pi x^*}{2}\right)\exp(-k^2 \pi^2 t^*),$$

where $x^*=\frac{x}{L}$ and $t^*=\frac{ct}{4L^2}$. We obviously can't compute the terms for values of $k$ all the way to infinity; we must choose a large enough number. As is usually done in python, we want to avoid using for loops and will use vectorization with ``numpy``'s ``meshgrid`` function.

The code below defines a function for you to obtain the analytical solution at a given position $x$, time $t$, and problems of the parameter $L, c,$ and $p_o$.


In [ ]:
def analytical_solution(x, t, L, c, p0=1, n=1e3):
    xstar = x/L
    tstar = c * t / (4 * L * L)
    k = np.arange(int(n))[1::2]

    # returns a tuple of shape (nk, nx, nt)
    xx, kk, tt = np.meshgrid(xstar, k, tstar)

    sinus = np.sin(0.5 * kk * np.pi * xx)
    exponent = np.exp(-kk * kk * np.pi * np.pi * tt)
    summed = np.sum(4 / (kk * np.pi) * sinus * exponent, axis=0)

    return p0 * summed

analytical_solution(x, 0, L, 1).shape

In the following, we compare the analytical solution for our problem parameters to the numerical solution obtained.

In [ ]:
nlines = 5 # Number of time steps we want to plot

# We choose to plot the solution at the beginning end and on three intermediate steps.
for i, ti in enumerate(np.linspace(1, len(t) - 1, nlines).astype(int)):

    # we plot the analytical solution first as a line    
    l, = plt.plot(x[:-1] + dx, analytical_solution(x[:-1] + dx, t[ti], L, c), label=f'{t[ti]:.2f}')

    # we then plot the numerical solution on top as only points
    plt.plot(x[:-1] + dx, heads[ti], c=l.get_color(), ls='none', marker='.', markevery=0.1, ms=10, mec='k')

# Visualization of the solution
plt.legend(title='time (s)', loc='center left', bbox_to_anchor=(1, 0.5), frameon=False)
plt.xlim(0, 1)
plt.xlabel('x (m)')
plt.ylabel('pressure (m)')

It is further possible to plot the solution in a time space plot where the color indicates the heads/pressure. 

In [ ]:
plt.imshow(heads, extent=[x[0], x[-1], t[0], t[-1]], origin='lower', cmap='Blues',)
plt.xlabel('x (m)')
plt.ylabel('t (s)')
cb = plt.colorbar()
plt.title('pressure (m)', loc='left')

Now let us perform an error estimate. For this we first need to obtain the analytical heads at all positions where we have the numerical solution. We then calculate the error in percent.

In [ ]:
# We get the analytical heads
# complete below
# anal_heads = analytical_solution(     ).T

# and calculate the error in percent
# complete below
# error = 

In [ ]:
# The following line is only to obtain the limits of the colorbar.
vmax = np.quantile(np.abs(error), 0.99)

# We visualize the error in the same way as we had visualized the solution on the domain.
plt.imshow(error, extent=[x[0], x[-1], t[0], t[-1]], origin='lower', cmap='coolwarm', vmin=-vmax, vmax=vmax)
plt.xlabel('x (m)')
plt.ylabel('t (s)')
cb = plt.colorbar()
plt.title('relative error (%)', loc='left')

Finally we can also animate the evolution of the pressure along the domain. Note that you might have to perform a `pip install` of the following libraries first:
    - `PyQt5`
    - `PyQt6`
    - `PySide6`

In [ ]:
pip install PyQt5 PyQt6 PySide6 

In [ ]:
# interactive python (iPython) magic to instruct jupyter how to show the plots with respect to the code cells
%matplotlib qt

# We predefine i as our iterable
i = 0

# Instantiation of the axis and figure
fig, ax = plt.subplots()

# The axis and text
l, = ax.plot(x[:-1] + dx, heads[i], c='k')
text = ax.set_title(f't = {t[i]:.2f}', loc='left')
ax.set_ylim(0, 1.05)
ax.set_xlim(0, 1)
ax.set_xlabel('x')
ax.set_ylabel('h', rotation=0, va='bottom')

# We need a function to update the plot
def update(i):
    # Change the y data
    l.set_ydata(heads[i])
    # Change the label to the new time.
    text.set_text(f't = {t[i]:.2f}')

    return l, text

anim = FuncAnimation(fig, update, frames=np.arange(0, len(t), 1), interval=25)

# interactive python (iPython) magic to instruct jupyter how to show the plots with respect to the code cells
%matplotlib inline
